In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bertopic import BERTopic
import pandas as pd 
from transformers.pipelines import pipeline
import os


# Loading articles

In [3]:
df_articles = pd.read_parquet("../files/bbc_news.parquet", engine="pyarrow")
df_articles.head(1)


,title,published_date,authors,description,section,content,link,top_image
0,Saido Berahino: Stoke complete deal to sign We...,2017-01-21,None,Stoke sign West Brom striker Saido Berahino fo...,None,Last updated on .From the section Football\n\n...,http://www.bbc.co.uk/sport/football/38696547,https://ichef.bbci.co.uk/onesport/cps/624/cpsp...


In [4]:
docs = df_articles['content'].tolist()

# Training Bertopic

In [6]:
##  min with mixedbread embeddings

# https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1?library=transformers

embedding_model = pipeline("feature-extraction", model="mixedbread-ai/mxbai-embed-large-v1")
topic_model = BERTopic(embedding_model=embedding_model)


In [7]:
topics, probs = topic_model.fit_transform(docs)

# Pushing model to Hugging Face 


In [ ]:
topic_model.push_to_hf_hub('CarlosMorales/bbc_news_bertopic', token=os.getenv("HF_TOKEN"))

# Finding articles about conflicts 

In [ ]:
topic_model.find_topics("Ukraine war", top_n=5)

AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
similar_topics, similarity = topic_model.find_topics("Ukraine war", top_n=5)
topic_model.get_representative_docs(similar_topics[0])

AttributeError: 'NoneType' object has no attribute 'keys'